In [23]:
import numpy as np
from scipy.io import loadmat
import pandas as pd
from sklearn.metrics import  accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,Dense,Dropout,Normalization,BatchNormalization,LayerNormalization
from tensorflow.keras.layers import LSTM,GRU,Embedding,Reshape
from tensorflow.keras.layers import Conv2D,MaxPooling2D,AveragePooling2D,Flatten
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam,Adamax
from tensorflow.keras.regularizers import l1,l2

In [13]:
# Load EEG data
x_train = loadmat('.../x_train')['x_train']
y_train = loadmat('.../y_train')['y_train'].T
x_test = loadmat('.../x_test')['x_test']
y_test = loadmat('.../y_test')['y_test'].T

# Allocate 10% of train data to validation set
x_val = x_train[-701:,:,:]
y_val = y_train[-701:,:]
x_train = x_train[:-701,:,:]
y_train = y_train[:-701,:]

In [22]:
# Print shapes of train and test sets
print(f'x_train shape:{x_train.shape},\
      \ny_train shape:{y_train.shape},\
      \nx_val shape:{x_val.shape},\
      \ny_val shape:{y_val.shape},\
      \nx_test shape:{x_test.shape},\
      \ny_test shape:{y_test.shape}\n')

# List the classes that exist within the dataset
print(f'Classes: {np.unique(y_train)}')
print('Seizure Types: \n1: for Complex Partial Seizures, 2: for Electrographic Seizures,\
       \n3: for Video-detected Seizures with no visual change over EEG, 0: for Normal data')

x_train shape:(6310, 19, 500),      
y_train shape:(6310, 1),      
x_val shape:(701, 19, 500),      
y_val shape:(701, 1),      
x_test shape:(779, 19, 500),      
y_test shape:(779, 1)

Classes: [0 1 2 3]
Seizure Types: 
1: for Complex Partial Seizures, 2: for Electrographic Seizures,       
3: for Video-detected Seizures with no visual change over EEG, 0: for Normal data


In [18]:
# Build the neural network model

tf.random.set_seed(1234)                                   # For consistent results

model = Sequential([
    Input((x_train.shape[1],x_train.shape[2],1)),          # Input layer for EEG data: (electrodes, time points, channels)

    Conv2D(8, (1,250), activation='relu', padding='same'), # Kernel length set to half the sampling rate of the data
    BatchNormalization(),                                  # Normalize activations to stabilize training
    AveragePooling2D((1,4)),                               # Downsample feature maps to reduce computational complexity and prevent overfitting
    Dropout(0.2),                                          # Include dropout to prevent overfitting

    Conv2D(16,(1,62), activation='relu', padding='same'),  # Additional convolutional layers to capture hierarchical features
    BatchNormalization(),
    AveragePooling2D((1,2)),
    Dropout(0.2),

    Conv2D(32,(1,31), activation='relu', padding='same'),
    BatchNormalization(),
    AveragePooling2D((1,2)),
    Dropout(0.2),

    Reshape((x_train.shape[1],-1)),                        # Reshape to prepare for LSTM layer
    LSTM(units=128, activation='relu'),

    Dense(4)
])

In [19]:
# Compile and fit the model

model.compile(
    loss=SparseCategoricalCrossentropy(from_logits=True),
    optimizer = Adamax(0.001),
    metrics = ['accuracy']
)

model.fit(
    x_train,y_train,
    epochs = 20,
    verbose = 0,
    validation_data=(x_val,y_val)
)

train_loss, train_accuracy = model.evaluate(x_train, y_train)
val_loss, val_accuracy = model.evaluate(x_val, y_val)
test_loss, test_accuracy = model.evaluate(x_test, y_test)

print(f'Train Loss: {train_loss:.2f}, Validation Loss: {val_loss:.2f},  Test Loss: {test_loss:.2f}')
print(f'Train Accuracy: {train_accuracy*100:.2f}%, Validation Accuracy: {val_accuracy*100:.2f}%, Test Accuracy: {test_accuracy*100:.2f}%')

198/198 ━━━━━━━━━━━━━━━━━━━━ 21s 104ms/step - accuracy: 0.9781 - loss: 0.0589
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 104ms/step - accuracy: 0.9275 - loss: 0.1975
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 97ms/step - accuracy: 0.9358 - loss: 0.2335
Train Loss: 0.06, Validation Loss: 0.20,  Test Loss: 0.26
Train Accuracy: 97.94%, Validation Accuracy: 93.15%, Test Accuracy: 92.43%


In [20]:
# Evaluate the model with other metrics

# Train data
y_train_predict_ = model.predict(x_train)
y_train_predict = np.argmax(tf.nn.softmax(y_train_predict_), axis=1)

# Test data
y_test_predict_ = model.predict(x_test)
y_test_predict = np.argmax(tf.nn.softmax(y_test_predict_), axis=1)

# Define the metrics and corresponding values
metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score']
train_values = [accuracy_score(y_train, y_train_predict) * 100,
                precision_score(y_train, y_train_predict, average='macro') * 100,
                recall_score(y_train, y_train_predict, average='macro') * 100,
                f1_score(y_train, y_train_predict, average='macro') * 100]
test_values = [accuracy_score(y_test, y_test_predict) * 100,
               precision_score(y_test, y_test_predict, average='macro') * 100,
               recall_score(y_test, y_test_predict, average='macro') * 100,
               f1_score(y_test, y_test_predict, average='macro') * 100]

# Create DataFrame
results_df = pd.DataFrame({
    'Metric:': metrics,
    'Train:': train_values,
    'Test:': test_values
})

# Print DataFrame
print("\nTrain and Test Data Results -")
print(results_df.round(2).to_string(index=False))

198/198 ━━━━━━━━━━━━━━━━━━━━ 20s 98ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 84ms/step

Train and Test Data Results -
  Metric:  Train:  Test:
 Accuracy   97.94  92.43
Precision   98.61  94.06
   Recall   98.60  93.08
 F1 Score   98.60  93.55
